In [1]:
import numpy as np
import pandas as pd
import datetime, pytz, calendar, urllib2, json

In [2]:
target_channels = ["Benzene","Toluene","Xylene","Hydrogen_Sulfide","m_p_Xylene",
                   "o_Xylene","Black_Carbon", "Ethylbenzene","Sulfur_Dioxide","voc","dust"]

In [14]:
def dateTimeToPacific(dt):
    pacific = pytz.timezone("US/Pacific")
    dt = pacific.localize(dt)
    return dt

In [45]:
#returns start and end timestamps of provided date
def getEpochTimeBounds(d, duration=1):
    dt = dateTimeToPacific(datetime.datetime(d.year,d.month,d.day))

    start = calendar.timegm(dt.utctimetuple())
    end = calendar.timegm((dt + datetime.timedelta(days=duration)).utctimetuple())
    return {'start' : start, 'end': end}
getEpochTimeBounds(datetime.date(2017,2,1))

{'end': 1486022400, 'start': 1485936000}

In [92]:
def makeDataFrameFromEsdr(feed, channel, timeOptions={}):
    if timeOptions.get('bounds') == None:
        duration = timeOptions.get('duration') or 1
        bounds = getEpochTimeBounds(timeOptions.get('day'), duration)
    else:
        bounds = timeOptions.get('bounds')
    url = "https://esdr.cmucreatelab.org/api/v1/feeds/%s/channels/%s/export?from=%s&to=%s&format=json" % (feed, channel, bounds['start'], bounds['end'])
    try:
        r = json.loads(urllib2.urlopen(url).read())
        print "loaded " + str(len(r['data'])) + " data points for feed " + feed + ", channel " + channel + ", time " + str(bounds['start'])
    except:
        print "error loading data from ESDR: feed " + feed + ", channel " + channel + ", time " + str(bounds['start'])
    cols = [name.split('.')[2] for name in r['channel_names']]
    cols.insert(0,'Time')
    df = pd.DataFrame(r["data"],columns=cols).set_index(['Time'])
    return df
df = makeDataFrameFromEsdr("4914","Benzene",{'day':datetime.date(2017,2,1)})
1487274113

loaded 1338 data points for feed 4914, channel Benzene, time 1485936000


1487274113

In [83]:
def maxHourlyAverage(df, feed):
    def avg(x, delta):
        ser = df.iloc[(df.index >= x - delta) & (df.index <= x + delta), 0]
        return ser.mean()
    
    healthLimit = 1
    df['avg'] = pd.Series(data = df.index, index = df.index).apply(lambda x: avg(x,delta=1800))
    maxAvg = df.nlargest(1,'avg')
    
    halfHourInSecs = 30 * 60
    hourStart = dateTimeToPacific(datetime.datetime.utcfromtimestamp(maxAvg.index[0] - halfHourInSecs)).strftime('%I:%M%p')
    hourEnd = dateTimeToPacific(datetime.datetime.utcfromtimestamp(maxAvg.index[0] + halfHourInSecs)).strftime('%I:%M%p')
    
    #get wind data for hour with highest average
    bounds = {'start':maxAvg.index[0] - 1800,'end':maxAvg.index[0] + 1800}
    wind = makeDataFrameFromEsdr(feed,"Wind_Direction,Wind_Speed_MPH",{'bounds':bounds})
    
    #break into quadrants and select the prevailing one
    quads = [0,90,180,270,360]
    quad_names = ['NE','SE','SW','NW']
    wind['Compass_Dir'] = pd.cut(wind['Wind_Direction'],quads,labels=quad_names)
    direction = wind.groupby('Compass_Dir').sum().nlargest(1,'Wind_Speed_MPH').index[0]
    print "The max hourly average was " + str(maxAvg.avg.item()) + " from " + hourStart + " to " + hourEnd +" and the prevailing wind direction was towards " + direction
    return maxAvg
maxHourlyAverage(df, "4914")
#pd.value_counts(wind['Compass_Dir'])
#sort into 4 groups by quadrant; quadrant with the most values is what ill display? should I also factor in wind speed? (quadrant with the highest sum of speeds?)
#wind
#df.sort_values('avg',ascending=False)
#df2.loc[df2['avg'] > 1].apply(lambda x: df.loc[(df.index >= x -  )])

loaded 53 data points for feed 4914, channel Wind_Direction,Wind_Speed_MPH, time 1485966000
The max hourly average was 1.19320754717 from 04:20PM to 05:20PM and the prevailing wind direction was towards NW


,Benzene,avg
Time,,
1485967800,1.26,1.193208


In [75]:
##dateTimeToPacific(datetime.datetime.utcfromtimestamp(1485965400 + (30*60))).s
#pacific = pytz.timezone("US/Pacific")
#pacific.astimezone(datetime.datetime.utcfromtimestamp(1485965400 + (30*60)))


In [93]:
def calcDailyMean(df, chemical, nd=0):
    if nd == 0:
        return df[chemical].mean()
    else:
        #substitute readings of 0 for the passed-in non-detect value
        #(which should represent that chemicals' detection limit)
        return df.replace(0.0,nd)[chemical].mean()
calcDailyMean(df,"Benzene", 0.5)

0.562167414050822

In [87]:
#total time, in hours, that a detection was present of given chemical or aggregated set of chemicals
def calcHoursDetected(df, chemical):
    detected = df.loc[df[chemical] > 0, [chemical]]
    return len(detected) / float(60) #each reading represents 1 minute
calcHoursDetected(df, "Benzene")

4.883333333333334

In [88]:
#total time, in hours, that detection was greater than health threshold of given chemical
def calcHoursAboveHealthLimit(df, chemical, limit):
    detected = df.loc[df[chemical] > limit, [chemical]]
    return len(detected) / float(60) #each reading represents 1 minute
calcHoursAboveHealthLimit(df, "Benzene", 1)

1.3166666666666667

In [12]:
detected = df.loc[df['val'] > 0, ['val']]
len(detected)

504

In [42]:
df2 = set()
delta = 1800
x = 1486021320
ser = df.loc[(df.index >= x - delta) & (df.index <= x + delta)]
#pd.merge(ser, df2, how='outer')
#set(ser.index).union(df2)